In [ ]:
# Standardbibliotheken
import sys

# Drittanbieter Libs
import numpy as np
import matplotlib.pyplot as plt
from PIL import ImageDraw, ImageFont, Image, ImageDraw
from pathlib import Path

# eigene Module
sys.path.append("../src")
from utils import luminance, clustersize, getGlyph

glyph_size = 20 #!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!! Glyphengröße

# Schriftart auswählen
def load_monospace_font(size: int) -> ImageFont.FreeTypeFont:
    local_path = Path.cwd().parent / "assets" / "Fira_Code" / "FiraCode-VariableFont_wght.ttf"
    if local_path.exists():
        print(f"Lade lokale Schrift: {local_path}")
        return ImageFont.truetype(str(local_path), size=size)
    else:
        print("Schriftart nicht gefunden, fallback …")
        try:
            return ImageFont.truetype("DejaVuSansMono.ttf", size=size)
        except Exception:
            return ImageFont.load_default()
        
font = load_monospace_font(glyph_size-1)

# Ordner festlegen
input_dir = Path("./SEQUENZ")         # hier liegen deine Eingangs-Bilder
output_dir = Path("./SEQUENZ_ERGEBNIS")        # hier sollen die Ergebnisse hin
output_dir.mkdir(exist_ok=True)

Lade lokale Schrift: c:\Users\Anna\SynologyDrive\Arbeit\Freelance\glyphgen\assets\Fira_Code\FiraCode-VariableFont_wght.ttf


In [86]:
image_files = list(input_dir.glob("*.png"))
print(f"{len(image_files)} Bilder gefunden.")

0 Bilder gefunden.


In [87]:
for img_path in image_files:
    print(f"Bearbeite {img_path.name} ...")
    
    # 1) Bild laden
    img = Image.open(img_path).convert("RGB")
    pixels = np.array(img)

    # Bildhöhe und -weite auslesen
    height, width = pixels.shape[:2]
    print(f'Bildhöhe = {height}')
    print(f'Bildbreite = {width}')

    # berechnent die Größe der Cluster, damit diese auch gleich verteilt sind und kein Rand entsteht
    cluster_y, cluster_x, cluster_height, cluster_width = clustersize(
        glyph_size, height, width
    )

    cluster_y, cluster_x = int(cluster_y), int(cluster_x)

    # 2D-Matrix für Glyphen (Liste von Listen)
    glyphen_matrix = [["" for _ in range(cluster_x)] for _ in range(cluster_y)]

    for i in range(cluster_y):
        for j in range(cluster_x):
            helligkeiten_added = 0.0

            # Blockbereich in Pixelkoordinaten
            row_start = i * cluster_height
            row_end   = (i + 1) * cluster_height
            col_start = j * cluster_width
            col_end   = (j + 1) * cluster_width

            # Durchschnittshelligkeit des Blocks
            
            cluster_helligkeit_durchschnitt = luminance(tuple(pixels[(int((row_start+row_end)/2)), int((col_start+col_end)/2)]))

            # Glyph bestimmen und in die Matrix schreiben
            glyph_temp = getGlyph(cluster_helligkeit_durchschnitt)
            glyphen_matrix[i][j] = glyph_temp

    # Grid-Bild kopieren um drauf zu malen
    overlay = Image.new("RGB", img.size, color=(0, 0, 0))
    draw = ImageDraw.Draw(overlay)

    plt.imshow(img)
    plt.title("Beispielbild ohne Grid")
    plt.axis("off")
    plt.show()
    for i in range(cluster_y):
        for j in range(cluster_x):
            ch = glyphen_matrix[i][j] or " "

            x0, y0 = j*cluster_width, i*cluster_height
            x1, y1 = (j+1)*cluster_width, (i+1)*cluster_height
            cx, cy = (x0 + x1) // 2, (y0 + y1) // 2

            # Glyph ins Zentrum zeichnen
            draw.text((cx, cy), ch, font=font,
                    fill=(0, 0, 0),
                    stroke_width=1, stroke_fill=(255, 255, 255),
                    anchor="mm")

    plt.figure(figsize=(8, 8 * img.height / img.width))
    plt.imshow(overlay)
    plt.axis("off")
    plt.title("Glyphen über den Zellen (mit kleinem Abstand)")
    plt.show()
        # 6) Ergebnis abspeichern
    out_path = output_dir / f"{img_path.stem}_glyphs.png"
    overlay.save(out_path)